In [2]:
# A code that splits reactants and products and gives the count of atoms
#and you have to run all the cells at the same time because the some of the functions are at the bottom
import os 
reactions_file = os.path.join('reactions.txt')


with open(reactions_file, 'r') as f:
    lines = f.readlines()

reactions = [line.strip() for line in lines]
f.close()

for reaction in reactions:
    reactants_together = reaction.split(" -> ")[0]
    products_together = reaction.split(" -> ")[1]
    reactants = reactants_together.split(" + ")
    products = products_together.split(" + ")
    print(reaction)
    #print('reactants: ', reactants)
    #print('products: ', products)
    for reactant in reactants:
        (n,mol) = stoichiometric_coefficient(reactant)
        #print("The coefficient for reactant",mol,"is",n)
    for product in products:
        (n,mol) = stoichiometric_coefficient(product)
        #print("The coefficient for product",mol,"is",n)
        
    allreactants = {}
    allproducts = {}
    for reactant in reactants:
        (n,mol) = stoichiometric_coefficient(reactant)
    #    print (n,mol)
        atoms = count_atoms(mol)
    #    print(atoms)
        for atom in atoms.keys():
    #        print(atom)
            if atom in allreactants.keys():
                allreactants[atom] += n * atoms[atom]
            else:
                allreactants[atom] = n * atoms[atom]
    for product in products:
        (n,mol) = stoichiometric_coefficient(product)
    #    print (n,mol)
        atoms = count_atoms(mol)
    #    print(atoms)
        for atom in atoms.keys():
    #        print(atom)
            if atom in allproducts.keys():
                allproducts[atom] += n * atoms[atom]
            else:
                allproducts[atom] = n * atoms[atom]
    print('reactants atomic count: ', allreactants)
    print('products atomic count: ', allproducts)
    #if allreactants == allproducts:
        #print ("Yeah, this is balanced")
    #else:
        #print ("Nope, it is not balanced")
        
        
    
    

         

C2H5OH + O2 -> CO2 + H2O


NameError: name 'stoichiometric_coefficient' is not defined

In [ ]:
# a code that gives the coefficient of the molecule 
def stoichiometric_coefficient(molecule):
    position = 0
    coefficient = ''
    while position < len(molecule):
        if molecule[position].isdigit():
            coefficient += molecule[position]
        else:
            break
        position += 1
    if coefficient == '':
        just_molecule = molecule
        return (1,just_molecule)
    else:
        just_molecule = molecule[len(coefficient):]
        return (int(coefficient),just_molecule)

In [ ]:
# the atomic _masses of the atoms involved in the reaction
atomic_masses = {}
atomic_masses['H'] = 1
atomic_masses['He'] = 4
atomic_masses['C'] = 12
atomic_masses['O'] = 16
print(atomic_masses)
print(list(atomic_masses.keys()))

In [ ]:
# a code that counts the number of atoms in a molecule 
def count_atoms(molecule):
    atoms = {}
    current_atom = ''
    current_number = ''
    ion = False
    for x in molecule:
        if x.isupper():
# x is a capital letter - start of a new atom
            if current_atom != '':
                if current_number == '': 
                    if current_atom in atoms.keys():
                        atoms[current_atom] += 1
                    else:
                        atoms[current_atom] = 1
                else:
                    if current_atom in atoms.keys():
                        atoms[current_atom] += int(current_number)
                    else:
                        atoms[current_atom] = int(current_number)
                current_atom = ''
                current_number = ''
            current_atom += x
        elif x.islower():
# x is a lowercase letter - second part of an atom
            current_atom += x
        elif x.isdigit() and not ion:
# x is a number - part of the coefficient
            current_number += x
        elif x in ['-', '+']:
            ion = True

    if current_atom != '':
        if current_number == '':
            if current_atom in atoms.keys():
                atoms[current_atom] += 1
            else:
                atoms[current_atom] = 1
        else:
            if current_atom in atoms.keys():
                atoms[current_atom] += int(current_number)
            else:
                atoms[current_atom] = int(current_number)
    return atoms

In [3]:
# a code that balances the chemical equation
# the problem with my code is that when it reads the first equation, because the next reaction has charges it breaks. 
import re
from sympy import Matrix, lcm
elementList=[]
elementMatrix=[]




def addToMatrix(element, index, count, side):
    if(index == len(elementMatrix)):
       elementMatrix.append([])
       for x in elementList:
            elementMatrix[index].append(0)
    if(element not in elementList):
        elementList.append(element)
        for i in range(len(elementMatrix)):
            elementMatrix[i].append(0)
    column=elementList.index(element)
    elementMatrix[index][column]+=count*side
    
def findElements(segment,index, multiplier, side):
    elementsAndNumbers=re.split('([A-Z][a-z]?)',segment)
    i=0
    while(i<len(elementsAndNumbers)-1):#last element always blank
          i+=1
          if(len(elementsAndNumbers[i])>0):
            if(elementsAndNumbers[i+1].isdigit()):
                count=int(elementsAndNumbers[i+1])*multiplier
                addToMatrix(elementsAndNumbers[i], index, count, side)
                i+=1
            else:
                addToMatrix(elementsAndNumbers[i], index, multiplier, side)        
    
def compoundDecipher(compound, index, side):
    segments=re.split('(\([A-Za-z0-9]*\)[0-9]*)',compound)    
    for segment in segments:
        if segment.startswith("("):
            segment=re.split('\)([0-9]*)',segment)
            multiplier=int(segment[1])
            segment=segment[0][1:]
        else:
            multiplier=1
        findElements(segment, index, multiplier, side)
            
for i in range(len(reactants)):
    compoundDecipher(reactants[i],i,1)
for i in range(len(products)):
    compoundDecipher(products[i],i+len(reactants),-1)
elementMatrix = Matrix(elementMatrix)
elementMatrix = elementMatrix.transpose()
solutions=elementMatrix.nullspace()[0]
multiplication = lcm([val.q for val in solutions])
solutions = multiplication*solutions
coEffi=solutions.tolist()
output=""
for i in range(len(reactants)):
    output+=str(coEffi[i][0])+reactants[i]
    if i<len(reactants)-1:
       output+=" + "
output+=" -> "
for i in range(len(products)):
   output+=str(coEffi[i+len(reactants)][0])+products[i]
   if i<len(products)-1:
       output+=" + "
print(output)

1C2H5OH + 3O2 -> 2CO2 + 3H2O
